In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [75]:
# Clean data, examine dfs

# train data
# c = train_df.columns.values.tolist()
# clean = []
# for col in c:
#  if train_df[col].dtype == "object":
#    clean.append(col)
# print(clean)
# train_df.describe
# test_df.shape
# train_X = train_df.drop([train_df[1], train_df[2])
test_df.iloc[1]
# train_df['loan_status']

#drop index columns, and results columns
train_df1 = train_df.drop(['Unnamed: 0', 'index'], axis=1)
test_df1 =test_df.drop(['Unnamed: 0', 'index'], axis=1)
print(train_df1.shape)
print(test_df1.shape)

(12180, 84)
(4702, 84)


In [89]:
# Convert categorical data to numeric and separate target feature for test data
test_df_dummies = pd.get_dummies(test_df1)
test_df_X = test_df_dummies.drop(['loan_status_low_risk', 'loan_status_high_risk'], axis=1)
print(test_df_X.shape)
train_df_y = train_df_dummies['loan_status_low_risk']


(4702, 91)


In [90]:
# Convert categorical data to numeric and separate target feature for train data
train_df_dummies = pd.get_dummies(train_df1)
# train_df_dummies.columns.values.tolist()
# train_df_dummies['loan_status_low_risk']
train_df_X = train_df_dummies.drop(['loan_status_low_risk', 'loan_status_high_risk'], axis=1)
print(train_df_X.shape)
train_df_y = train_df_dummies['loan_status_low_risk']
# train_df_y.head()

(12180, 92)


In [106]:
# add missing dummy variables to testing set
#what columns is missing?
# cat = train_df_X.columns.values.tolist()
# cleanc1 = []
# for col in cat:
#  if train_df_X[col].dtype == "uint8":
#    cleanc1.append(col)
# print(cleanc1)

# cat = train_df_X.columns.values.tolist()
# cleanc = []
# for col in cat:
#  if test_df_X[col].dtype == "uint8":
# #    cleanc.append(col)
#    print(col,test_df_X[col].dtype )
    
#missing columns 'debt_settlement_flag_Y'
# pd.get_dummies(df).reindex(columns = cats, fill_value=0)

test_df_X1 = pd.get_dummies(test_df_X).reindex(columns = ['debt_settlement_flag_Y'], fill_value=0)
test_df_X2 = pd.concat([test_df_X,test_df_X1], axis=1, join="inner")
test_df_X2.shape

(4702, 92)

In [ ]:
# Train the Logistic Regression model on the unscaled data and print the model score

In [ ]:
# Train a Random Forest Classifier model and print the model score

In [ ]:
# Scale the data

In [ ]:
# Train the Logistic Regression model on the scaled data and print the model score

In [ ]:
# Train a Random Forest Classifier model on the scaled data and print the model score